In [12]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/mac/Documents/Documents/ID2223_Scalable/lab1_new/Air_Quality_Prediction
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">Part 02: Vienna Multi-Sensor Daily Feature Pipeline</span>

## 🗒️ This notebook is divided into the following sections:
1. Load Vienna Sensors Configuration
2. Download Today's Air Quality Data (all sensors)
3. Download Weather Forecast Data (all sensor locations)
4. Feature Group Insertion

## 📊 Covered Sensors (9 active sensors in Vienna):
This pipeline collects daily data from all active Vienna air quality sensors:
- Kendlerstraße 40 (ID: 2850)
- Hausgrundweg 23 (ID: 2855)
- Allgemeines Krankenhaus Ostringweg (ID: 14537)
- Gaudenzdorfer Gürtel (ID: 2857)
- Belgradplatz (ID: 2870)
- Floridsdorf Gerichtsgasse (ID: 4738)
- Taborstraße (ID: 2860)
- Wehlistraße 366 (ID: 4736)
- Josef Redl Gasse 2 (ID: 4739)

__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [13]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
import sys
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Load Vienna Sensors Configuration from Hopsworks </span>

This notebook retrieves the Vienna sensors configuration from Hopsworks Secrets.

The configuration was saved by the backfill notebook (`1_vienna_multi_sensor_backfill.ipynb`) and contains:
- All sensor locations and metadata
- API URLs for each sensor
- Sensor coordinates for weather data

We will process **all active sensors** in a single daily run.


In [14]:
# Login to Hopsworks
print("🔐 Logging in to Hopsworks...")
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# Retrieve API key and Vienna sensors configuration
print("🔑 Retrieving secrets...")
try:
    AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
    vienna_config_str = secrets.get_secret("VIENNA_SENSORS_CONFIG").value
    vienna_config = json.loads(vienna_config_str)
except Exception as e:
    print(f"❌ Error retrieving secrets: {e}")
    print("Make sure you have run 1_vienna_multi_sensor_backfill.ipynb first!")
    sys.exit(1)

# Extract configuration
country = vienna_config['country']
city = vienna_config['city']
all_sensors = vienna_config['sensors']
active_sensors = [s for s in all_sensors if s.get('status') == 'active']

today = datetime.date.today()

print(f"✅ Successfully logged in to Hopsworks")
print(f"\n📍 City: {city}, {country}")
print(f"📅 Today's date: {today}")
print(f"📡 Active sensors: {len(active_sensors)}")
print(f"\n🔍 Active sensors list:")
for i, sensor in enumerate(active_sensors, 1):
    print(f"  {i}. {sensor['name']} ({sensor['street']})")

🔐 Logging in to Hopsworks...
2025-11-16 12:21:42,290 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-16 12:21:42,296 INFO: Initializing external client
2025-11-16 12:21:42,296 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 12:21:43,817 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1298582
🔑 Retrieving secrets...
✅ Successfully logged in to Hopsworks

📍 City: Vienna, Austria
📅 Today's date: 2025-11-16
📡 Active sensors: 9

🔍 Active sensors list:
  1. Kendlerstraße 40 (Umspannwerk) (Kendlerstrasse-40)
  2. Hausgrundweg 23, Gstr. 254 (Hausgrundweg-23)
  3. Allgemeines Krankenhaus, Ostringweg (AKH-Ostringweg)
  4. Umspannwerk Gaudenzdorfer Gürtel (Gaudenzdorfer-Guertel)
  5. Belgradplatz (Südostecke), Gstr.Nr. 816 (Belgradplatz)
  6. Floridsdorf, Gerichtsgasse 1a (Floridsdorf-Gerichtsgasse)
  7. Ecke Taborstraße - Glockengasse (Taborstrasse)
  8. Wehlistraße 366, Gstr.Nr.2157 (Wehlistrasse-366)
  9. Schafbergbad, Josef Redl Gasse 2 (Josef-Redl-Gasse)


---

## <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [15]:
# Retrieve Vienna-specific feature groups
print("🔄 Retrieving Feature Groups...")

air_quality_fg = fs.get_feature_group(
    name='air_quality_vienna',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_vienna',
    version=1,
)

print(f"✅ Feature Group: {air_quality_fg.name} (v{air_quality_fg.version})")
print(f"   Primary key: {air_quality_fg.primary_key}")
print(f"✅ Feature Group: {weather_fg.name} (v{weather_fg.version})")
print(f"   Primary key: {weather_fg.primary_key}")

🔄 Retrieving Feature Groups...
✅ Feature Group: air_quality_vienna (v1)
   Primary key: ['country', 'city', 'street']
✅ Feature Group: weather_vienna (v1)
   Primary key: ['city', 'street']


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>

We will fetch today's PM2.5 measurements from **all active sensors** in Vienna.


In [16]:
# Fetch today's PM2.5 data for all active sensors
all_aq_data = []

print(f"🌫 Fetching today's PM2.5 data for {len(active_sensors)} sensors...\n")

for i, sensor in enumerate(active_sensors, 1):
    sensor_url = sensor['aqicn_url']
    sensor_street = sensor['street']
    
    print(f"  {i}/{len(active_sensors)}. {sensor['name']}")
    print(f"      Street: {sensor_street}")
    
    try:
        # Fetch today's PM2.5 data for this sensor
        aq_sensor_df = util.get_pm25(
            sensor_url, 
            country, 
            city, 
            sensor_street, 
            today, 
            AQICN_API_KEY
        )
        
        if len(aq_sensor_df) > 0 and not pd.isna(aq_sensor_df['pm25'].iloc[0]):
            all_aq_data.append(aq_sensor_df)
            pm25_value = aq_sensor_df['pm25'].iloc[0]
            print(f"      ✅ PM2.5: {pm25_value:.1f}")
        else:
            print(f"      ⚠️  No data available")
        
    except Exception as e:
        print(f"      ❌ Error: {str(e)}")
    
    print()
    
    # Add small delay to avoid rate limiting
    if i < len(active_sensors):
        time.sleep(2)

# Combine all sensor data
if len(all_aq_data) > 0:
    aq_today_df = pd.concat(all_aq_data, ignore_index=True)
    
    print(f"\n✅ Successfully fetched PM2.5 data:")
    print(f"   Total records: {len(aq_today_df)}")
    print(f"   Sensors with data: {aq_today_df['street'].nunique()}")
    print(f"\n📊 Today's Air Quality Data:")
    display(aq_today_df)
else:
    print("❌ No PM2.5 data available for any sensor!")
    sys.exit(1)

🌫 Fetching today's PM2.5 data for 9 sensors...

  1/9. Kendlerstraße 40 (Umspannwerk)
      Street: Kendlerstrasse-40
      ✅ PM2.5: 50.0

  2/9. Hausgrundweg 23, Gstr. 254
      Street: Hausgrundweg-23
      ✅ PM2.5: 50.0

  3/9. Allgemeines Krankenhaus, Ostringweg
      Street: AKH-Ostringweg
      ✅ PM2.5: 50.0

  4/9. Umspannwerk Gaudenzdorfer Gürtel
      Street: Gaudenzdorfer-Guertel
      ✅ PM2.5: 46.0

  5/9. Belgradplatz (Südostecke), Gstr.Nr. 816
      Street: Belgradplatz
      ✅ PM2.5: 42.0

  6/9. Floridsdorf, Gerichtsgasse 1a
      Street: Floridsdorf-Gerichtsgasse
      ✅ PM2.5: 53.0

  7/9. Ecke Taborstraße - Glockengasse
      Street: Taborstrasse
      ✅ PM2.5: 38.0

  8/9. Wehlistraße 366, Gstr.Nr.2157
      Street: Wehlistrasse-366
      ✅ PM2.5: 46.0

  9/9. Schafbergbad, Josef Redl Gasse 2
      Street: Josef-Redl-Gasse
      ✅ PM2.5: 53.0


✅ Successfully fetched PM2.5 data:
   Total records: 9
   Sensors with data: 9

📊 Today's Air Quality Data:


,pm25,country,city,street,date,url
0,50.0,Austria,Vienna,Kendlerstrasse-40,2025-11-16,https://api.waqi.info/feed/@2850
1,50.0,Austria,Vienna,Hausgrundweg-23,2025-11-16,https://api.waqi.info/feed/@2855
2,50.0,Austria,Vienna,AKH-Ostringweg,2025-11-16,https://api.waqi.info/feed/@14537
3,46.0,Austria,Vienna,Gaudenzdorfer-Guertel,2025-11-16,https://api.waqi.info/feed/@2857
4,42.0,Austria,Vienna,Belgradplatz,2025-11-16,https://api.waqi.info/feed/@2870
5,53.0,Austria,Vienna,Floridsdorf-Gerichtsgasse,2025-11-16,https://api.waqi.info/feed/@4738
6,38.0,Austria,Vienna,Taborstrasse,2025-11-16,https://api.waqi.info/feed/@2860
7,46.0,Austria,Vienna,Wehlistrasse-366,2025-11-16,https://api.waqi.info/feed/@4736
8,53.0,Austria,Vienna,Josef-Redl-Gasse,2025-11-16,https://api.waqi.info/feed/@4739


In [17]:
# Display DataFrame info
print("\n📋 Air Quality DataFrame Info:")
aq_today_df.info()


📋 Air Quality DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     9 non-null      float32       
 1   country  9 non-null      object        
 2   city     9 non-null      object        
 3   street   9 non-null      object        
 4   date     9 non-null      datetime64[ns]
 5   url      9 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 528.0+ bytes


---

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data for each sensor location</span>

We will fetch 7-day weather forecasts for **each sensor's specific location** (not just city center).

This ensures accurate weather data for each area of Vienna.

In [18]:
# Fetch weather forecast for each sensor location
all_weather_data = []

print(f"🌦 Fetching 7-day weather forecasts for {len(active_sensors)} locations...")
print(f"   ⏰ Estimated time: ~{(len(active_sensors)-1) * 2 / 60:.1f} minutes\n")

for i, sensor in enumerate(active_sensors, 1):
    sensor_lat = sensor['latitude']
    sensor_lon = sensor['longitude']
    sensor_street = sensor['street']
    
    print(f"  {i}/{len(active_sensors)}. {sensor['name']}")
    print(f"      Coordinates: ({sensor_lat:.4f}, {sensor_lon:.4f})")
    
    try:
        # Get hourly forecast for this sensor location
        hourly_df = util.get_hourly_weather_forecast(city, sensor_lat, sensor_lon)
        hourly_df = hourly_df.set_index('date')
        
        # Extract daily forecast (noon time: 12:00)
        daily_df = hourly_df.between_time('11:59', '12:01')
        daily_df = daily_df.reset_index()
        daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
        daily_df['date'] = pd.to_datetime(daily_df['date'])
        daily_df['city'] = city
        daily_df['street'] = sensor_street  # Add sensor identifier
        
        all_weather_data.append(daily_df)
        print(f"      ✅ {len(daily_df)} daily forecasts")
        
    except Exception as e:
        print(f"      ❌ Error: {str(e)}")
    
    print()
    
    # Add delay to avoid rate limiting
    if i < len(active_sensors):
        time.sleep(2)

# Combine all weather data
if len(all_weather_data) > 0:
    weather_df = pd.concat(all_weather_data, ignore_index=True)
    
    print(f"\n✅ Successfully fetched weather forecasts:")
    print(f"   Total records: {len(weather_df)}")
    print(f"   Locations: {weather_df['street'].nunique()}")
    print(f"   Forecast days: {weather_df['date'].nunique()}")
    print(f"\n📊 Weather Forecast Data (sample):")
    display(weather_df.head(14))  # Show first 2 sensors (7 days each)
else:
    print("❌ No weather data available!")
    sys.exit(1)

🌦 Fetching 7-day weather forecasts for 9 locations...
   ⏰ Estimated time: ~0.3 minutes

  1/9. Kendlerstraße 40 (Umspannwerk)
      Coordinates: (48.2050, 16.3098)
Coordinates 48.25°N 16.25°E
Elevation 234.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
      ✅ 7 daily forecasts

  2/9. Hausgrundweg 23, Gstr. 254
      Coordinates: (48.2264, 16.4583)
Coordinates 48.25°N 16.5°E
Elevation 160.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
      ✅ 7 daily forecasts

  3/9. Allgemeines Krankenhaus, Ostringweg
      Coordinates: (48.2191, 16.3498)
Coordinates 48.25°N 16.25°E
Elevation 199.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
      ✅ 7 daily forecasts

  4/9. Umspannwerk Gaudenzdorfer Gürtel
      Coordinates: (48.1871, 16.3393)
Coordinates 48.0°N 16.5°E
Elevation 180.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
      ✅ 7 daily forecasts

  5/9. Belgradplatz (Südostecke), Gstr.Nr. 816
      Coordinates: (48.1744, 16.3614)
Coordi

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city,street
0,2025-11-16,12.05,0.0,8.396570,149.036316,Vienna,Kendlerstrasse-40
1,2025-11-17,9.55,2.8,13.324863,271.548126,Vienna,Kendlerstrasse-40
2,2025-11-18,5.10,0.0,17.015474,276.072357,Vienna,Kendlerstrasse-40
3,2025-11-19,4.75,0.0,15.038350,137.910919,Vienna,Kendlerstrasse-40
4,2025-11-20,7.05,0.0,10.538843,262.146790,Vienna,Kendlerstrasse-40
5,2025-11-21,3.10,0.1,12.727921,298.739685,Vienna,Kendlerstrasse-40
6,2025-11-22,2.70,0.1,4.394360,55.007900,Vienna,Kendlerstrasse-40
7,2025-11-16,11.50,0.0,8.311245,162.349792,Vienna,Hausgrundweg-23
8,2025-11-17,9.65,3.2,5.815978,248.198532,Vienna,Hausgrundweg-23
9,2025-11-18,5.85,0.0,13.202726,281.003479,Vienna,Hausgrundweg-23


In [19]:
# Display DataFrame info
print("\n📋 Weather Forecast DataFrame Info:")
weather_df.info()


📋 Weather Forecast DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         63 non-null     datetime64[ns]
 1   temperature_2m_mean          63 non-null     float32       
 2   precipitation_sum            63 non-null     float32       
 3   wind_speed_10m_max           63 non-null     float32       
 4   wind_direction_10m_dominant  63 non-null     float32       
 5   city                         63 non-null     object        
 6   street                       63 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(2)
memory usage: 2.6+ KB


---

## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

We will insert today's data into the Vienna-specific Feature Groups.

In [20]:
# Insert today's air quality data
print(f"📤 Inserting {len(aq_today_df)} air quality records...")
print(f"   From {aq_today_df['street'].nunique()} sensors")
print(f"   Date: {today}")

air_quality_fg.insert(aq_today_df)

print("✅ Air quality data inserted successfully!")

📤 Inserting 9 air quality records...
   From 9 sensors
   Date: 2025-11-16
2025-11-16 12:22:32,758 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1298582/fs/1286214/fg/1703402


Uploading Dataframe: 100.00% |██████████| Rows 9/9 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_vienna_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1298582/jobs/named/air_quality_vienna_1_offline_fg_materialization/executions
✅ Air quality data inserted successfully!


In [21]:
# Insert weather forecast data
print(f"\n📤 Inserting {len(weather_df)} weather forecast records...")
print(f"   From {weather_df['street'].nunique()} sensor locations")
print(f"   Forecast days: {weather_df['date'].nunique()}")

weather_fg.insert(weather_df, wait=True)

print("✅ Weather forecast data inserted successfully!")


📤 Inserting 63 weather forecast records...
   From 9 sensor locations
   Forecast days: 7
2025-11-16 12:22:46,995 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1298582/fs/1286214/fg/1703403


Uploading Dataframe: 100.00% |██████████| Rows 63/63 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_vienna_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1298582/jobs/named/weather_vienna_1_offline_fg_materialization/executions
2025-11-16 12:23:04,597 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-16 12:23:07,785 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-16 12:24:59,885 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-16 12:25:00,064 INFO: Waiting for log aggregation to finish.
2025-11-16 12:25:32,478 INFO: Execution finished successfully.
✅ Weather forecast data inserted successfully!


In [22]:
# Print detailed summary statistics
print("=" * 80)
print("📊 DAILY FEATURE PIPELINE SUMMARY")
print("=" * 80)
print(f"\n🌍 Location: {city}, {country}")
print(f"📅 Date: {today}")

print(f"\n📡 Sensors processed: {len(active_sensors)}")
print(f"   Sensors with PM2.5 data: {aq_today_df['street'].nunique()}")
print(f"   Sensors with weather forecasts: {weather_df['street'].nunique()}")

print(f"\n📊 Data Statistics:")
print(f"   Air Quality records inserted: {len(aq_today_df)}")
print(f"   Weather Forecast records inserted: {len(weather_df)}")
print(f"   Forecast days per sensor: {weather_df.groupby('street')['date'].nunique().values[0] if len(weather_df) > 0 else 0}")

print(f"\n🌫 PM2.5 Summary (Today):")
if len(aq_today_df) > 0:
    print(f"   Average: {aq_today_df['pm25'].mean():.1f}")
    print(f"   Min: {aq_today_df['pm25'].min():.1f} ({aq_today_df.loc[aq_today_df['pm25'].idxmin(), 'street']})")
    print(f"   Max: {aq_today_df['pm25'].max():.1f} ({aq_today_df.loc[aq_today_df['pm25'].idxmax(), 'street']})")

print(f"\n✅ Feature Groups updated:")
print(f"   1. {air_quality_fg.name} (v{air_quality_fg.version})")
print(f"   2. {weather_fg.name} (v{weather_fg.version})")

print(f"\n🔗 Hopsworks Project: {project.name}")
print("=" * 80)
print("\n✅ Pipeline completed successfully!")


📊 DAILY FEATURE PIPELINE SUMMARY

🌍 Location: Vienna, Austria
📅 Date: 2025-11-16

📡 Sensors processed: 9
   Sensors with PM2.5 data: 9
   Sensors with weather forecasts: 9

📊 Data Statistics:
   Air Quality records inserted: 9
   Weather Forecast records inserted: 63
   Forecast days per sensor: 7

🌫 PM2.5 Summary (Today):
   Average: 47.6
   Min: 38.0 (Taborstrasse)
   Max: 53.0 (Floridsdorf-Gerichtsgasse)

✅ Feature Groups updated:
   1. air_quality_vienna (v1)
   2. weather_vienna (v1)

🔗 Hopsworks Project: Air_Quality_ID2223

✅ Pipeline completed successfully!


---

## <span style="color:#ff5f27;"> 🎉 Daily Feature Pipeline Complete!</span>

Successfully completed the following:
- ✅ Retrieved Vienna sensors configuration from Hopsworks Secrets
- ✅ Fetched today's PM2.5 data from all active sensors in Vienna
- ✅ Fetched 7-day weather forecasts for each sensor's specific location
- ✅ Inserted all data into Feature Store

### 📊 Key Features:
- **Location-specific weather data**: Each sensor gets weather forecast for its exact location (not just city center)
- **Multi-sensor support**: Handles all 9 active Vienna sensors in one run
- **Rate limiting**: Built-in delays to avoid API throttling
- **Error handling**: Gracefully handles individual sensor failures
- **Feature Groups**: `air_quality_vienna` and `weather_vienna`

### 🔄 This notebook should run daily to keep the Feature Store up-to-date!

You can schedule this using:
- GitHub Actions
- Cron jobs
- Apache Airflow
- Any Python orchestration tool

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from the feature groups and create training datasets within the feature store
